<a href="https://colab.research.google.com/github/yy0750/Study_ML_Project/blob/main/%EC%8A%A4%ED%85%8C%EC%9D%B4%EC%A7%80_1_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%9D%98_%EA%B8%B0%EB%B3%B8_%EA%B5%AC%EC%A1%B0%EC%99%80_%ED%8A%B9%EC%84%B1_%EC%9D%B4%ED%95%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 스테이지 1. 데이터의 기본 구조와 특성 이해
## 도입
안녕하세요, 데이터 사이언티스트 여러분!🙌

스테이지 1에서는 ‘데이터의 기본 구조와 특성 이해’에 중점을 둡니다.<br>
데이터 분석의 기초를 닦는 과정으로, 데이터를 불러오고, 체계적으로 확인하는 작업을 통해 데이터에 대한 깊이 있는 이해를 목표로 합니다.<br>
우리는 수치형 데이터와 범주형 데이터의 기술통계량을 확인하며, 범주형 특성의 고유값도 함께 분석할 것입니다.<br>
이 과정을 통해, 데이터의 전반적인 특성과 구조를 파악하고, 다음 스테이지에서의 분석 방향을 설정하는 데 도움을 얻을 수 있습니다.

## 학습 목표
- 데이터를 성공적으로 불러오고, 데이터의 기본적인 구조와 정보를 확인하기
- 수치형(Numerical) 데이터의 기술통계량을 분석하여 데이터의 중심 경향, 분산, 범위 등을 이해하기
- 범주형(categorical) 데이터의 기술통계량을 분석하여 각 범주의 빈도와 중요성을 파악하기
- 범주형 특성의 고유값을 확인하여 데이터의 다양성을 이해하기

# 1. 데이터 불러오기

In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

# 2. 데이터 확인하기

In [ ]:
display(train.head(5))
display(test.head(5))
display(submission.head(5))

# 3. 데이터 정보 확인

In [ ]:
train.info()

# 4. 수치형(numerical) 데이터 기술통계량 확인

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

description = train.describe().T

fig, axs = plt.subplots(figsize=(11, 3))
sns.heatmap(description, annot=True, fmt=".2f", cmap='coolwarm', linewidths=.5, ax=axs)
axs.set_title('Descriptive Statistics Heatmap')
plt.show()

# 5. 범주형(categorical) 데이터 기술통계량 확인


In [ ]:
train_drop = train.drop(['ID'],axis = 1)

train_drop.describe(include=['object']).T

# 6. 범주형 피처(feature) 고유값 출력

In [ ]:
categorical_Eng = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country']
categorical_Kor = ['근로 형태', '교육', '결혼 상태', '직업', '가족 관계', '인종', '성별', '국적']

for Eng, Kor in zip(categorical_Eng, categorical_Kor ):
    unique_values = train[Eng].unique().tolist()
    print('\n {0} ({1})\n'.format(Eng, Kor), unique_values)

In [ ]:
# 데이터 분할 및 전처리
train_x = train.drop(columns=['ID', 'Income'])
train_y = train['Income']
test_x = test.drop(columns=['ID'])

In [ ]:
# 범주형 변수 수치화
from sklearn.preprocessing import LabelEncoder
import numpy as np

encoding_target = list(train_x.dtypes[train_x.dtypes == "object"].index)
for i in encoding_target:
    le = LabelEncoder()

    # train과 test 데이터셋에서 해당 열의 모든 값을 문자열로 변환
    train_x[i] = train_x[i].astype(str)
    test_x[i] = test_x[i].astype(str)

    le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])

    # test 데이터의 새로운 카테고리에 대해 le.classes_ 배열에 추가
    for case in np.unique(test_x[i]):
        if case not in le.classes_:
            le.classes_ = np.append(le.classes_, case)

    test_x[i] = le.transform(test_x[i])


test_x.head()

In [ ]:
# # train_x와 test_x에서 'Gender' 열 변환
# train_x['Gender'] = train_x['Gender'].map({'M': 1, 'F': 0})
# test_x['Gender'] = test_x['Gender'].map({'M': 1, 'F': 0})

# test_x.head()

In [ ]:
# 선형 회귀 모델 초기화 및 훈련
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(train_x, train_y)

In [ ]:
test

In [ ]:
from sklearn.metrics import mean_squared_error
# 테스트 데이터에 대한 예측
predictions = model.predict(test_x)

# RMSE 계산
rmse = mean_squared_error(test_y, predictions, squared=False)
rmse